In [10]:
import pandas as pd
import csv
import time
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import xgboost as xgb

In [24]:
paths = ['submission_baseline20.csv', 'submission_baseline20.csv', 'submission2_5_0.578.csv']

In [25]:
def read_sub(path, weight=1):
    return(
        pl.read_csv(path)
        .with_column(pl.col('labels').str.split(by=' '))
        .with_column(pl.lit(weight).alias('vote'))
        .explode('labels')
        .rename({'labels': 'aid'})
        .with_column(pl.col('aid').cast(pl.UInt32))
        .with_column(pl.col('vote').cast(pl.UInt8))
    )

In [26]:
subs = [read_sub(path) for path in paths]
subs[0].head()

session_type,aid,vote
str,u32,u8
"""12899779_click...",59625,1
"""12899779_carts...",59625,1
"""12899779_order...",59625,1
"""12899780_click...",1142000,1
"""12899780_click...",582732,1


In [27]:
subs = subs[0].join(subs[1], how='outer', on=['session_type', 'aid']).join(subs[2], how = 'outer', on=['session_type', 'aid'], suffix = '_right2')

In [28]:
subs = (subs
    .fill_null(0)
    .with_column((pl.col('vote') + pl.col('vote_right') + pl.col('vote_right2')).alias('vote_sum'))
    .drop(['vote', 'vote_right', 'vote_right2'])
    .sort(by='vote_sum')
    .reverse()
)

subs.head()

session_type,aid,vote_sum
str,u32,u8
"""13574184_order...",607738,3
"""13574184_order...",1180072,3
"""13574184_order...",1607333,3
"""13965939_carts...",621012,3
"""13253974_order...",1174724,3


In [30]:
subs

session_type,aid,vote_sum
str,u32,u8
"""13574184_order...",607738,3
"""13574184_order...",1180072,3
"""13574184_order...",1607333,3
"""13965939_carts...",621012,3
"""13253974_order...",1174724,3
"""13213945_carts...",73989,3
"""14427778_carts...",1170613,3
"""14427778_carts...",1290922,3
"""14062271_carts...",406829,3


In [29]:
%%time
preds = subs.groupby('session_type').agg([
    pl.col('aid').head(20).alias('labels')
])

preds = preds.with_column(pl.col('labels').apply(lambda lst: ' '.join([str(aid) for aid in lst])))

CPU times: total: 4min 59s
Wall time: 4min 39s


In [31]:
preds.write_csv('submission.csv')

In [49]:
preds.sort_values(by='session_type').reset_index()

AttributeError: 'DataFrame' object has no attribute 'sort_values'

In [44]:
preds['labels'][0]

'989071 1733323 1741080 1022850 1333526 1672161 345879 1748048 1638804 1879 941724 86364 527490 681237 616137 835817 973329 1200248 253576 383147'